In [1]:
import gdal
from matplotlib import pyplot as plt
import rasterio
from skimage.filters.rank import entropy
from skimage.morphology import disk
import cv2
import numpy as np
from skimage.util import img_as_ubyte

In [2]:
from skimage.exposure import adjust_gamma, adjust_sigmoid, adjust_log
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from numpy import save,load
from skimage.morphology import area_opening,area_closing,remove_small_holes,remove_small_objects,erosion,dilation,disk,convex_hull_object,square

# import dataset

In [3]:
# FOR LANDSAT 7
year = gdal.Open('KNP2008.tif')
%matplotlib qt5
band1 = year.GetRasterBand(1)
band2 = year.GetRasterBand(2)
band3 = year.GetRasterBand(3)
band4 = year.GetRasterBand(4)
band5 = year.GetRasterBand(5)
band6 = year.GetRasterBand(6)

band1_array = img_as_ubyte(band1.ReadAsArray())
band2_array = img_as_ubyte(band2.ReadAsArray())
band3_array = img_as_ubyte(band3.ReadAsArray())
band4_array = img_as_ubyte(band4.ReadAsArray())
band5_array = img_as_ubyte(band5.ReadAsArray())
band6_array = img_as_ubyte(band6.ReadAsArray())

year_name = '2011'

In [16]:
# FOR LANDSAT 8
year = gdal.Open('KNP2018.tif')

band1 = year.GetRasterBand(6)
band2 = year.GetRasterBand(5)
band3 = year.GetRasterBand(4)
band4 = year.GetRasterBand(3)
band5 = year.GetRasterBand(2)
band6 = year.GetRasterBand(1)

band1_array = img_as_ubyte(band1.ReadAsArray())
band2_array = img_as_ubyte(band2.ReadAsArray())
band3_array = img_as_ubyte(band3.ReadAsArray())
band4_array = img_as_ubyte(band4.ReadAsArray())
band5_array = img_as_ubyte(band5.ReadAsArray())
band6_array = img_as_ubyte(band6.ReadAsArray())

year_name = '2016'

# Sand Bed Extraction - Otsu Based

In [271]:
image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band1_array.astype('int')
image[:,:,1] = band3_array.astype('int')
image[:,:,2] = band6_array.astype('int')
gray = (0.2125*image[:,:,0] + 0.7154*image[:,:,1] + 0.0721*image[:,:,2]).astype('int')

In [272]:
plt.imshow(gray, cmap='gray')
plt.show()

In [273]:
th = threshold_otsu(gray)
print(th)

36


In [274]:
b_sand = dilation(erosion((gray>th)*1, disk(1)),disk(1))
b_sand =remove_small_holes(remove_small_objects(b_sand>0,150),150)

In [275]:
%matplotlib qt5
plt.imshow(b_sand, cmap = 'gray')
plt.suptitle('Sand mask (2008)')
plt.axis('off')
plt.show()

In [276]:
v_image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
v_image[:,:,0] = band6_array.astype('int')
v_image[:,:,1] = band4_array.astype('int')
v_image[:,:,2] = band2_array.astype('int')

In [277]:
edge_sand = cv2.Canny((b_sand*255).astype(np.uint8), 50, 150)

In [278]:
contours,hierarchy = cv2.findContours(edge_sand,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [279]:
k = cv2.drawContours(v_image.astype(int), contours, -1, (255, 0, 0), 1)

In [280]:
plt.imshow(k.astype(int), cmap = plt.cm.gray)
plt.show()

#### Graphs and Histogram 

In [295]:
plt.hist(gray.ravel(), bins = range(256))
plt.title('Kaziranga Image 2008')
plt.xlim(0,255)
plt.axvline(th, color='r', linestyle='dashed', linewidth=1)
plt.show()

### Trial

In [634]:
image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band1_array.astype('int')
image[:,:,1] = band3_array.astype('int')
image[:,:,2] = band6_array.astype('int')

In [635]:
gray = (0.2125*image[:,:,0] + 0.7154*image[:,:,1] + 0.0721*image[:,:,2]).astype('int')

In [298]:
enh_img_log = (adjust_log(gray,1)).astype('int')

In [299]:
plt.imshow(enh_img_log, cmap = plt.cm.jet)
plt.show()

In [550]:
th = threshold_otsu(gray)
print(th)
max_val = np.max(enh_img_log.ravel())
print(max_val)

55
233


In [551]:
plt.hist(gray.ravel(), bins = range(256))
plt.title('Kaziranga Image '+year_name)
plt.axvline(th, color='r', linestyle='dashed', linewidth=2)
plt.show()

In [365]:
b_sand = dilation(erosion((gray>th)*1, disk(1)),disk(1))

In [402]:
%matplotlib qt5
plt.imshow(b_sand, cmap = 'gray')
plt.suptitle('Sand mask - '+year_name)
plt.axis('off')
plt.show()

# Histograms of all bands

In [300]:
plt.hist(band1_array.ravel(), bins = range(256))
plt.title('Band 1 (Blue) - '+year_name)
plt.xlabel('I(pi) --------->')
plt.show()

In [301]:
fig,a =  plt.subplots(2,3)
a[0][0].set_title('Band 1 - Blue')
a[0][0].hist(band1_array.ravel(), bins = range(256))
a[0][0].set_xlabel('Intensity of Pixels ----->')
a[0][0].set_ylabel('Count(n) ----->')
a[0][0].set_xlim([0,255])

a[0][1].set_title('Band 2 - Green')
a[0][1].hist(band2_array.ravel(), bins = range(256))
a[0][1].set_xlabel('Intensity of Pixels --------------- >')
a[0][1].set_ylabel('Count(n) ----->')
a[0][1].set_xlim([0,255])

a[0][2].set_title('Band 3 - Red')
a[0][2].hist(band3_array.ravel(), bins = range(256))
a[0][2].set_xlabel('Intensity of Pixels --------------- >')
a[0][2].set_ylabel('Count(n) ----->')
a[0][2].set_xlim([0,255])

a[1][0].set_title('Band 4 - NIR')
a[1][0].hist(band4_array.ravel(), bins = range(256))
a[1][0].set_xlabel('Intensity of Pixels --------------- >')
a[1][0].set_ylabel('Count(n) ----->')
a[1][0].set_xlim([0,255])

a[1][1].set_title('Band 5 - SWIR1')
a[1][1].hist(band5_array.ravel(), bins = range(256))
a[1][1].set_xlabel('Intensity of Pixels --------------- >')
a[1][1].set_ylabel('Count(n) ----->')
a[1][1].set_xlim([0,255])

a[1][2].set_title('Band 6 - SWIR2')
a[1][2].hist(band6_array.ravel(), bins = range(256))
a[1][2].set_xlabel('Intensity of Pixels --------------- >')
a[1][2].set_ylabel('Count(n) ----->')
a[1][2].set_xlim([0,255])



plt.show()

In [302]:

plt.hist(image[:, :, 0].ravel(), bins = range(256), color = 'red', alpha = 0.5)
plt.hist(image[:, :, 1].ravel(), bins = range(256), color = 'Green', alpha = 0.5)
plt.hist(image[:, :, 2].ravel(), bins = range(256), color = 'Blue', alpha = 0.5)
plt.xlabel('Intensity Value')
plt.ylabel('Count')
plt.legend(['Red_Channel(Blue band)', 'Green_Channel(Red band)', 'Blue_Channel(SWIR2 band)'])
plt.show()

In [303]:
plt.hist(gray.ravel(), bins = range(256), color = 'gray', )
plt.axvline(th, color='r', linestyle='dashed', linewidth=3)
plt.xlabel('Intensity Value ------>')
plt.ylabel('Count ------>')
plt.legend(['Otsu Threshold '])
plt.show()

# River Extraction - Otsu Based

In [281]:
gray_river = (0.7875*band4_array + 0.2125*band5_array)

In [282]:
th_river = threshold_otsu(gray_river)
print(th_river)
#max_val = np.max(enh_img_log.ravel())
#print(max_val)

26.006787109374997


In [283]:
river_binary = area_opening(area_closing(gray_river<th_river))

In [284]:
river_mask_processed =remove_small_holes(remove_small_objects(river_binary,2500),2500)


In [285]:
plt.imshow(river_mask_processed,cmap=plt.cm.gray)
plt.show()

In [4]:
v_image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
v_image[:,:,0] = band6_array.astype('int')
v_image[:,:,1] = band4_array.astype('int')
v_image[:,:,2] = band2_array.astype('int')

In [287]:
edge_river = cv2.Canny((river_mask_processed*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(edge_river,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k = cv2.drawContours(v_image.astype(int), contours, -1, (255, 0, 0), 1)
plt.imshow(k.astype(int), cmap = plt.cm.gray)
plt.show()

#### histograms 

In [251]:
plt.hist(band4_array.ravel(), bins = range(256), color = 'brown', alpha = 0.5)
plt.hist(band5_array.ravel(), bins = range(256), color = 'orange', alpha = 0.6)
plt.xlabel('Intensity Value ---->')
plt.ylabel('Count ----->')
plt.legend(['NIR', 'SWIR1'])
plt.show()

In [252]:
plt.hist(gray_river.ravel(), bins = range(256))
plt.axvline(th_river, color='r', linestyle='dashed', linewidth=3)
plt.xlabel('Intensity Value ---->')
plt.ylabel('Count ----->')
plt.xlim(0,255)
plt.legend(['Otsu Threshold'])
plt.show()

# Combined

In [288]:
final_vegetation = ((river_mask_processed + b_sand)>0)*1

In [289]:
plt.imshow(final_vegetation,cmap=plt.cm.gray)
plt.show()

In [290]:
edge_forest = cv2.Canny((final_vegetation*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(edge_forest,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k = cv2.drawContours(v_image.astype(int), contours, -1, (255, 0, 0), 1)
plt.imshow(k.astype(int), cmap = plt.cm.gray)
plt.show()

In [291]:
final_vegetation.shape

(853, 1798)

In [292]:
def Find_Kaziranga_Edge(bin_img):
    for col in range(bin_img.shape[1]-1,-1,-1):
        state = 1
        for row in range(bin_img.shape[0]-1,-1,-1):
            if bin_img[row][col] == 1:
                state = 0
            bin_img[row][col] = state
    return bin_img

In [293]:
plt.imshow(Find_Kaziranga_Edge(final_vegetation), cmap = plt.cm.gray)
plt.show()

In [294]:
signal_vegetation = np.array([ np.sum(final_vegetation[:,i]) for i in range(0,1798)])

In [295]:
signal_vegetation

array([ 46,  46,  46, ..., 447, 448, 448])

In [296]:
def Plot_signal_vegetation(sv):
    blank_img = np.zeros((853,1798))
    for col in range(blank_img.shape[1]-1,-1,-1):
        count = sv[col]
        for row in range(blank_img.shape[0]-1,-1,-1):
            if count > 0:
                count = count-1
                blank_img[row][col] = 1
    return blank_img

In [297]:
plt.imshow(Plot_signal_vegetation(signal_vegetation), cmap = 'gray')
plt.show()

# Refining Boundary

#### savitzky-golay filtering 

In [298]:
from scipy.signal import savgol_filter

In [299]:
signal_vegetation_processed = savgol_filter(signal_vegetation,153,3)
#signal_vegetation_processed2 = savgol_filter(signal_vegetation_processed,151,3)

In [300]:
processed_final_vegetation = Plot_signal_vegetation(signal_vegetation_processed)
plt.imshow(processed_final_vegetation, cmap = 'gray')
plt.show()

In [302]:
from numpy import save,load
save('Comparison_mask_2017.npy',processed_final_vegetation)

In [301]:
processed_final_vegetation_edge = cv2.Canny((processed_final_vegetation*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(processed_final_vegetation_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k = cv2.drawContours(v_image.astype(int), contours, -1, (255, 0, 0), 2)
plt.imshow(k.astype(int), cmap = plt.cm.gray)
plt.show()

#### Convert Annotation to mask 

kaziranga forest 2008 image ground truth marking

In [78]:
from skimage import io
moon = io.imread('KNP2018_ann.png')

In [79]:
boundary_2008_ann = (moon[:,:,0]==255) & (moon[:,:,1]==0)*1
mask_2008_ann = np.zeros(boundary_2008_ann.shape)
for col in range(boundary_2008_ann.shape[1]-1,-1,-1):
        state = 1
        for row in range(boundary_2008_ann.shape[0]-1,-1,-1):
            if boundary_2008_ann[row][col] == 1:
                state = 0
            mask_2008_ann[row][col] = state

plt.imshow(mask_2008_ann, cmap = 'gray')
plt.show()

In [1128]:
from numpy import save,load

save('KNP2008_actual_mask',mask_2008_ann)

kaziranga forest 2018 image ground truth marking

In [177]:
moon2 = io.imread('KNP2018_ann.png')
boundary_2018_ann = (moon2[:,:,0]==255) & (moon2[:,:,1]==0)*1
mask_2018_ann = np.zeros(boundary_2018_ann.shape)
for col in range(boundary_2018_ann.shape[1]-1,-1,-1):
        state = 1
        for row in range(boundary_2018_ann.shape[0]-1,-1,-1):
            if boundary_2018_ann[row][col] == 1:
                state = 0
            mask_2018_ann[row][col] = state

plt.imshow(mask_2018_ann, cmap = 'gray')
plt.show()

In [1130]:
save('KNP2018_actual_mask',mask_2018_ann)

#### Visualizing Refined Results 

In [178]:
def Convert_To_Signal(obtained_mask):
    
    signal_obtained = np.array([ np.sum(obtained_mask[:,i]) for i in range(0,1798)])
    
    return signal_obtained

In [179]:
def find_ws_ms(w1,w2,a_s,c_s,poly_ord):
    """
    Outputs: window sizes(in range w1 to w2)
          and corresponding MS error for each window size   
    """
    w = np.array(range(w1,w2+1,2))
    error = np.array([])
    for i in range(w1,w2+1,2):
        f_s = savgol_filter(c_s,i,poly_ord)
        mse = np.sum((a_s-f_s)**2)/a_s.shape[0]
        error = np.append(error,mse)
    
    return w,error

In [73]:
def Plot_Results_savgol(actual_mask, obtained_mask):
    """
    Obtained_mask: the noisy signal
    """
    actual_signal,obtained_signal = Convert_to_Signal(actual_mask,obtained_mask)
    
    # Window Size to be varied from w1 to w2 and polynomials from p1 to p2
    w1 = 3
    w2 = 200
    p1 = 0
    p2 = 9
    
    ws_ms,ms_ws = find_ws_ms(w1,w2,actual_signal,obtained_signal)
    po_ms,ms_po = find_po_ms(p1,p2,actual_signal,obtained_signal)
    
    ws_iou,iou_ws = find_ws_iou(w1,w2,actual_mask,obtained_mask)
    po_iou,iou_po = find_po_iou(p1,p2,actual_mask,obtained_mask)
    
    ws_dice,dice_ws = find_ws_dice(w1,w2,actual_mask,obtained_mask)
    po_dice,dice_po = find_po_dice(p1,p2,actual_mask,obtained_mask)
    
    # Plot 3 figures with the different windows and polynomials
    
    
    

In [180]:
a_msk = Convert_To_Signal(load('KNP2018_actual_mask.npy')) 


In [181]:
ext_msk = signal_vegetation
ext_msk

array([ 45,  46,  46, ..., 418, 418, 419])

In [182]:
wins0,errs0 = find_ws_ms(11,501,a_msk,ext_msk,0)
print('done 1')
wins1,errs1 = find_ws_ms(11,501,a_msk,ext_msk,1)
print('done 2')
wins2,errs2 = find_ws_ms(11,501,a_msk,ext_msk,2)
print('done 3')
wins3,errs3 = find_ws_ms(11,501,a_msk,ext_msk,3)
print('done 4')
wins4,errs4 = find_ws_ms(11,501,a_msk,ext_msk,4)
print('done 5')
wins5,errs5 = find_ws_ms(11,501,a_msk,ext_msk,5)
print('done 6')
wins6,errs6 = find_ws_ms(11,501,a_msk,ext_msk,6)
print('done 7')
wins7,errs7 = find_ws_ms(11,501,a_msk,ext_msk,7)
print('done 8')
wins8,errs8 = find_ws_ms(11,501,a_msk,ext_msk,8)

done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8


In [223]:

plt.plot(wins2,errs2, label = 'polynomial order 2')
plt.plot(wins3,errs3, label = 'polynomial order 3')
plt.plot(wins4,errs4, label = 'polynomial order 4')
plt.plot(wins5,errs5, label = 'polynomial order 5')
plt.legend()
plt.xlabel('Window Size --->')
plt.ylabel('Mean Square Error (MSE) --->')
plt.title('Mean Square Error Curves for Different Window Size and Polynomial Order')
plt.show()

In [227]:
errs4[189:199]

array([193.35268195, 193.24944654, 193.2437857 , 193.26474422,
       193.28742894, 193.3497446 , 193.45905197, 193.57693748,
       193.72162602, 193.88215183])

# Comparing boundary visually


In [48]:
from skimage import io

actual_bound = load('KNP2018_actual_mask.npy')
plt.imshow(actual_bound)
plt.show()

In [62]:
comp_img_bound = cv2.drawContours(v_image.astype(int), contours, -1, (255, 0, 255), 2)
plt.imshow(comp_img_bound.astype(int))
plt.show()

In [63]:
actual_bound_edge = cv2.Canny((actual_bound*255).astype(np.uint8), 50, 150)
contours2,hierarchy2 = cv2.findContours(actual_bound_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
final_comp = cv2.drawContours(comp_img_bound.astype(int), contours2, -1, (255, 255, 0), 2)
plt.imshow(final_comp.astype(int), cmap = plt.cm.gray)
plt.show()

# Projecting Unstable Areas

In [5]:
from skimage import io

forest_area_2008 = load('Comparison_mask_2008.npy')
forest_area_2009 = load('Comparison_mask_2009.npy')
forest_area_2014 = load('Comparison_mask_2014.npy')
forest_area_2018 = load('Comparison_mask_2018.npy')

In [9]:
plt.imshow(v_image.astype('int'))
plt.show()

In [17]:
knp_ann = io.imread('KNP2008_ann.png')

In [18]:
forest_area_2008_edge = cv2.Canny((forest_area_2008*255).astype(np.uint8), 50, 150)
contours1,hierarchy1 = cv2.findContours(forest_area_2008_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
res1 = cv2.drawContours(knp_ann.astype(int), contours1, -1, (0, 255, 255), 1)
plt.imshow(res1.astype(int), cmap = plt.cm.gray)
plt.show()

In [101]:
forest_area_2009_edge = cv2.Canny((forest_area_2009*255).astype(np.uint8), 50, 150)
contours2,hierarchy2 = cv2.findContours(forest_area_2009_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
res2 = cv2.drawContours(res1.astype(int), contours2, -1, (0, 255, 0), 2)
plt.imshow(res2.astype(int), cmap = plt.cm.gray)
plt.show()

In [102]:
forest_area_2014_edge = cv2.Canny((forest_area_2014*255).astype(np.uint8), 50, 150)
contours3,hierarchy3 = cv2.findContours(forest_area_2014_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
res3 = cv2.drawContours(res2.astype(int), contours3, -1, (255, 0, 0), 2)
plt.imshow(res3.astype(int), cmap = plt.cm.gray)
plt.show()

In [103]:
forest_area_2018_edge = cv2.Canny((forest_area_2018*255).astype(np.uint8), 50, 150)
contours4,hierarchy4 = cv2.findContours(forest_area_2018_edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
res4 = cv2.drawContours(res3.astype(int), contours4, -1, (255, 0, 255), 2)


In [104]:
plt.imshow(res4.astype(int), cmap = plt.cm.gray)
plt.show()

# Erosion Analysis

In [814]:
region1_2008 = v_image[216:450,1200:1520,:]

In [817]:
region1_2018 = v_image[216:450,1200:1520,:]

In [821]:
plt.imshow(region1_2008.astype('int'))
plt.show()

In [822]:
plt.imshow(region1_2018.astype('int'))
plt.show()

In [825]:
erosion = (forest_area_2008[216:450,1200:1520]-forest_area_2018[216:450,1200:1520])>0

In [826]:
plt.imshow(erosion.astype('int'), cmap = 'gray')
plt.show()

In [828]:
erosion*1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [830]:
np.sum(erosion*1)

6007

In [834]:
plt.imshow(forest_area_2018[216:450,1200:1520], cmap = 'gray')
plt.show()

In [835]:
np.sum(forest_area_2008[216:450,1200:1520]*1)

48183.0

In [836]:
np.sum(forest_area_2018[216:450,1200:1520]*1)

42176.0

# Creating Graph on Erosion per year and table

Scale 1 pixel = 900 sq.m                                                                 
Heads : year, forest cover area, erosion in that year, cumulative erosion since 2008                                           
critical area slice : [216:450, 1200:1520]

In [305]:
forest_area_2008 = load('Comparison_mask_2008.npy')
forest_area_2009 = load('Comparison_mask_2009.npy')
forest_area_2010 = load('Comparison_mask_2010.npy')
forest_area_2011 = load('Comparison_mask_2011.npy')
forest_area_2013 = load('Comparison_mask_2013.npy')
forest_area_2014 = load('Comparison_mask_2014.npy')
forest_area_2015 = load('Comparison_mask_2015.npy')
forest_area_2016 = load('Comparison_mask_2016.npy')
forest_area_2017 = load('Comparison_mask_2017.npy')
forest_area_2018 = load('Comparison_mask_2018.npy')

In [306]:
erosion8to9 = (forest_area_2008[216:450,1200:1520]-forest_area_2009[216:450,1200:1520])>0
erosion9to10 = (forest_area_2009[216:450,1200:1520]-forest_area_2010[216:450,1200:1520])>0
erosion10to11 = (forest_area_2010[216:450,1200:1520]-forest_area_2011[216:450,1200:1520])>0
erosion11to13 = (forest_area_2011[216:450,1200:1520]-forest_area_2013[216:450,1200:1520])>0
erosion13to14 = (forest_area_2013[216:450,1200:1520]-forest_area_2014[216:450,1200:1520])>0
erosion14to15 = (forest_area_2014[216:450,1200:1520]-forest_area_2015[216:450,1200:1520])>0
erosion15to16 = (forest_area_2015[216:450,1200:1520]-forest_area_2016[216:450,1200:1520])>0
erosion16to17 = (forest_area_2016[216:450,1200:1520]-forest_area_2017[216:450,1200:1520])>0
erosion17to18 = (forest_area_2017[216:450,1200:1520]-forest_area_2018[216:450,1200:1520])>0

In [316]:
np.sum(erosion17to18)

1136

In [336]:
x = np.array([2008,2009,2010,2011,2013,2014,2015,2016,2017,2018])
y = np.array([0,383,995,782,1208,933,541,911,1543,1136])*0.03*0.03


In [337]:
plt.plot(x,y,linestyle='--', marker='s', color='b')
plt.xlabel('Year --->')
plt.ylabel('Erosion (sq.km) --->')
plt.show()

In [329]:
y

array([0.3447, 0.8955, 0.7038, 1.0872, 0.8397, 0.4869, 0.8199, 1.3887,
       1.0224])

In [354]:
y2 = np.array([0,383,1235,1712,2613,3264,3411,4094,5152,6007])*0.03*0.03

In [355]:
plt.plot(x,y2,linestyle='--', marker='s', color='black')
plt.xlabel('Year --->')
plt.ylabel('Cumulative Erosion since 2008 (sq.km) --->')
plt.show()

In [332]:
x.shape

(9,)

In [334]:
y2.shape

(10,)

In [335]:
y.shape

(9,)

In [342]:
erosion8to9 = (forest_area_2008[216:450,1200:1520]-forest_area_2009[216:450,1200:1520])>0
erosion8to10 = (forest_area_2008[216:450,1200:1520]-forest_area_2010[216:450,1200:1520])>0
erosion8to11 = (forest_area_2008[216:450,1200:1520]-forest_area_2011[216:450,1200:1520])>0
erosion8to13 = (forest_area_2008[216:450,1200:1520]-forest_area_2013[216:450,1200:1520])>0
erosion8to14 = (forest_area_2008[216:450,1200:1520]-forest_area_2014[216:450,1200:1520])>0
erosion8to15 = (forest_area_2008[216:450,1200:1520]-forest_area_2015[216:450,1200:1520])>0
erosion8to16 = (forest_area_2008[216:450,1200:1520]-forest_area_2016[216:450,1200:1520])>0
erosion8to17 = (forest_area_2008[216:450,1200:1520]-forest_area_2017[216:450,1200:1520])>0
erosion8to18 = (forest_area_2008[216:450,1200:1520]-forest_area_2018[216:450,1200:1520])>0

In [351]:
np.sum(erosion8to18)

6007

In [356]:
y2

array([0.    , 0.3447, 1.1115, 1.5408, 2.3517, 2.9376, 3.0699, 3.6846,
       4.6368, 5.4063])